In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("diabetes_data.csv")


In [3]:
data.replace(
    "?", np.nan, inplace=True
)  # replace '?' to np.nan to understand missing value better


#### diag_1, diag_2, diag_3 variables : 

The dataset contained upto three diagnoses for a given patient (primary (diag_1), secondary(diag_2) and additional(diag_3)). However, each of these had 700–900 unique ICD codes  
You can collapsed these diagnosis codes into 9 disease categories . These 9 categories include Circulatory, Respiratory, Digestive, Diabetes, Injury, Musculoskeletal, Genitourinary, Neoplasms, and Others. 

* You can convert all 3 into 9 categories each. however primary diagnosis is enough for this study.It is upto you!!

ICD9 code conversion reference (Read about it online to understand more)

ICD-9 CM Category ICD-9 CM Codes Frequency % of Total Circulatory System 390-459.9 562 30.6% Respiratory System 460-519.9 258 13.8% Digestive System 520-579.9 223 12.2% Neoplasms 140-239.9 123 6.7% Genitourinary System 580-629.9 114 6.2% Signs, Symptoms, and 780-799.9 103 5.6% Ill-Defined Conditions Mental Disorders 290-319 85 4.6% Musculoskeletal System 710-739.9 83 4.5% and Connective Tissue Injury and Poisoning 800-999.9 83 4.5% Endocrine, Nutritional, 240-279.9 69 3.8% Metabolic ...

In [4]:
def convert_diag_codes(code):
    #     print (code, type(code))
    if pd.isnull(code):
        return "Other"
    elif ("V" in code) or ("E" in code):
        return "Other"
    else:
        code = float(code)
        if (code >= 390) and (code < 460) or (np.floor(code) == 785):
            return "Circulatory"
        elif (code >= 460) and (code < 520) or (np.floor(code) == 786):
            return "Respiratory"
        elif (code >= 520) and (code < 580) or (np.floor(code) == 787):
            return "Digestive"
        elif code == 250:
            return "Diabetes"
        elif (code >= 800) and (code < 1000):
            return "Injury"
        elif (code >= 710) and (code < 740):
            return "Musculoskeletal"
        elif (code >= 580) and (code < 630) or (np.floor(code) == 788):
            return "Genitourinary"
        elif (code >= 140) and (code < 240):
            return "Neoplasms"
        else:
            return "Other"


In [5]:
data["diag_1"] = data["diag_1"].apply(lambda x: convert_diag_codes(x))


### Convert age to numeric

In [6]:
age_dict = {
    "[0-10)": 5,
    "[10-20)": 15,
    "[20-30)": 25,
    "[30-40)": 35,
    "[40-50)": 45,
    "[50-60)": 55,
    "[60-70)": 65,
    "[70-80)": 75,
    "[80-90)": 85,
    "[90-100)": 95,
}
data["age"] = data["age"].map(age_dict)
data["age"] = data["age"].astype("int64")


### Convert 2 value categories into binary variable

In [7]:
data["change"] = data["change"].replace("Ch", 1)
data["change"] = data["change"].replace("No", 0)
data["gender"] = data["gender"].replace("Male", 1)
data["gender"] = data["gender"].replace("Female", 0)
data["diabetesMed"] = data["diabetesMed"].replace("Yes", 1)
data["diabetesMed"] = data["diabetesMed"].replace("No", 0)


### Merge categories to one/two level less - Use domain knowledge

In [8]:
data["A1Cresult"] = data["A1Cresult"].replace(">7", "Abnormal")
data["A1Cresult"] = data["A1Cresult"].replace(">8", "Abnormal")
data["A1Cresult"] = data["A1Cresult"].replace("Norm", "Normal")
data["A1Cresult"] = data["A1Cresult"].replace("None", "Not tested")
data["max_glu_serum"] = data["max_glu_serum"].replace(">200", "Abnormal")
data["max_glu_serum"] = data["max_glu_serum"].replace(">300", "Abnormal")
data["max_glu_serum"] = data["max_glu_serum"].replace("Norm", "Normal")
data["max_glu_serum"] = data["max_glu_serum"].replace("None", "Not tested")


### Merge some categories together - By studying other variables

Admission source, admission type and discharge disposition can help us to understand what theses admission type id corresponds to. For example, admission types 1, 2 and 7 correspond to Emergency, Urgent Care and Trauma, and thus were combined into a single category as these are all non-elective situations. 

In [9]:
data["admission_type_id"] = data["admission_type_id"].replace(2, 1)
data["admission_type_id"] = data["admission_type_id"].replace(7, 1)
data["admission_type_id"] = data["admission_type_id"].replace(6, 5)
data["admission_type_id"] = data["admission_type_id"].replace(8, 5)


### Cautions!! Some columns looks numerical but they are not... Convert them into string
example: admission_type_id, discharge_disposition_id

In [10]:
data["admission_type_id"] = data["admission_type_id"].map(str)
data["discharge_disposition_id"] = data["discharge_disposition_id"].map(str)


### Remove duplicate patients; This is creating 
* Either simply remove them
* Find a way to deal with them - domain knowledge

In [11]:
data = data.drop_duplicates(subset=["patient_nbr"], keep="first")


### Drop columns where values are missing >=40% or have same values, doesn't need for analysis

* You can add or drop more variables from below list

In [12]:
# data.drop(['patient_nbr', 'diag_2', 'diag_3', 'encounter_id', 'admission_source_id'], axis = 1, inplace=True)  #, 'weight','payer_code','medical_specialty', 'citoglipton', 'examide'


### Check missing value

In [13]:
def cal_missing_val(df):
    data_dict = {}
    for col in df.columns:
        data_dict[col] = (df[col].isnull().sum() / df.shape[0]) * 100
    return pd.DataFrame.from_dict(
        data_dict, orient="index", columns=["MissingValueInPercentage"]
    )


cal_missing_val(data)


,MissingValueInPercentage
encounter_id,0.0
patient_nbr,0.0
race,0.0
gender,0.0
age,0.0
admission_type_id,0.0
discharge_disposition_id,0.0
admission_source_id,0.0
time_in_hospital,0.0
num_lab_procedures,0.0


### Fill missing value for 'Race'
https://scikit-learn.org/stable/modules/impute.html

In [14]:
## you can be creative here  -- this is for demo purpose
data["race"].replace(np.nan, "Missing", inplace=True)


### Select categorical columns to get_dummies

I have selected 27 columns here..

In [15]:
categorical_cols = [
    "race",
    "admission_type_id",
    "discharge_disposition_id",
    "diag_1",
    "max_glu_serum",
    "A1Cresult",
    "metformin",
    "repaglinide",
    "nateglinide",
    "chlorpropamide",
    "glimepiride",
    "acetohexamide",
    "glipizide",
    "glyburide",
    "tolbutamide",
    "pioglitazone",
    "rosiglitazone",
    "acarbose",
    "miglitol",
    "troglitazone",
    "tolazamide",
    "insulin",
    "glyburide-metformin",
    "glipizide-metformin",
    "glimepiride-pioglitazone",
    "metformin-pioglitazone",
]


### Convert to dummies

In [16]:
data_dummies = pd.get_dummies(data[categorical_cols], drop_first=True)


In [17]:
data_dummies.head(5)


,race_Asian,race_Caucasian,race_Hispanic,race_Other,admission_type_id_3,admission_type_id_4,admission_type_id_5,discharge_disposition_id_10,discharge_disposition_id_11,discharge_disposition_id_12,...,troglitazone_Steady,tolazamide_Steady,insulin_No,insulin_Steady,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,metformin-pioglitazone_Steady
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


### Change readmitted (response variable) into binary variable

In [18]:
readmit_dict = {">30": 1, "<30": 1, "NO": 0}
data["readmitted"] = data["readmitted"].map(readmit_dict)


### Merge dummies with numerical cols

In [19]:
noncategorical_cols = [col for col in data.columns if col not in categorical_cols]
noncategorical_cols


['encounter_id',
 'patient_nbr',
 'gender',
 'age',
 'admission_source_id',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'change',
 'diabetesMed',
 'readmitted']

In [20]:
data_preprocess = pd.concat([data[noncategorical_cols], data_dummies], axis=1)


In [21]:
data_preprocess.shape  # this is a dataset ready for further analysis


(68630, 106)

In [23]:
data_preprocess.to_csv(
    "diabetes_data_preprocess.csv", index=False
)  ## R user can use this file


#### I think 100-120 columns are pretty good to work with. You can run this analysis both in R/Python very smoothly